In [1]:
from typing import Protocol, TypeVar
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, StringType
from typedspark import (
    Schema,
    DataSet,
    Column,
    DataSetExtends,
    DataSetImplements,
    transform_to_schema,
    create_empty_dataset,
)


class Person(Schema):
    name: Column[StringType]
    age: Column[LongType]


class Job(Schema):
    role: Column[StringType]


class Age(Schema, Protocol):
    age: Column[LongType]


spark = SparkSession.builder.getOrCreate()

person = create_empty_dataset(spark, Person)
job = create_empty_dataset(spark, Job)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/14 20:09:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
def get_age(df: DataSetExtends[Age]) -> DataSet[Age]:
    return transform_to_schema(df, Age)


get_age(person)
try:
    get_age(job)  # linting error: Job is not a subtype of Age
except:
    pass

In [11]:
T = TypeVar("T", bound=Schema)


def birthday(
    df: DataSetImplements[Age, T],
) -> DataSet[T]:
    return transform_to_schema(
        df,
        df.typedspark_schema,  # type: ignore
        {
            Age.age: Age.age + 1,
        },
    )


res = birthday(person)
try:
    res = birthday(job)  # linting error: Job is not a subtype of Age
except:
    pass